In [1]:
import sys
sys.path.insert(0, './../Models')

from mlp_mixer import MLPMixer
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

/Users/pushkalm11/miniconda3/envs/MLP_mixer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net = MLPMixer(in_channels = 3,
               dim = 768,
               num_classes = 21843,
               patch_size = 16,
               image_size = 224,
               depth = 12,
               token_dim = 384,
               channel_dim = 3072)

In [3]:
def convert_keys(state_dict):
    keys = state_dict.keys()
    new_keys = []
    new_state_dict = {}

    for key in keys:
        new_key = key.replace("/", ".")
        new_key = new_key.replace("MixerBlock_", "MixerBlock.")
        new_key = new_key.replace("channel_mixing.Dense_0", "channel_mixing.1.net.0")
        new_key = new_key.replace("channel_mixing.Dense_1", "channel_mixing.1.net.3")
        new_key = new_key.replace("token_mixing.Dense_0", "token_mixing.2.net.0")
        new_key = new_key.replace("token_mixing.Dense_1", "token_mixing.2.net.3")
        new_key = new_key.replace("LayerNorm_0", "token_mixing.0")
        new_key = new_key.replace("LayerNorm_1", "channel_mixing.0")
        new_key = new_key.replace("scale", "weight")
        new_key = new_key.replace("kernel", "weight")
        new_key = new_key.replace("stem", "stem.0")
        new_key = new_key.replace("head", "head.0")
        new_key = new_key.replace("pre_head.0_layer_norm", "pre_head_layer_norm")
        new_keys.append(new_key)
    
    for (key, new_key) in zip(keys, new_keys):
        new_state_dict[new_key] = torch.tensor(state_dict[key], dtype = torch.float32).T
    return new_state_dict

In [4]:
google_weights = np.load("./../Weights/imagenet21k-Mixer-B_16.npz", allow_pickle = True)
new_weights = convert_keys(google_weights)
net.load_state_dict(new_weights, strict = False)

/var/folders/0d/v0qrrszd3395__m529hc_8800000gn/T/ipykernel_3752/1611322628.py:23: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3618.)
  new_state_dict[new_key] = torch.tensor(state_dict[key], dtype = torch.float32).T


<All keys matched successfully>